---
# **機械学習特論　第6回課題**
## **【アダブースト・勾配ブースティング】Iris・MNIST・Fashion-MNISTの分類**
---

In [15]:
# ライブラリの読み込み
# %pip install --upgrade pip
# %pip install numpy
# %pip install matplotlib
# %pip install pandas
# %pip install openpyxl
# %pip install sympy
# %pip install scipy
# %pip install re
# %pip install jaconv
# %pip install scikit-learn
# %pip install statsmodels
# %pip install seaborn
# %pip install pmdarima
# %pip install kneed
# %pip install scikit-learn-intelex
# %pip install lightgbm

from matplotlib.font_manager import FontProperties
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import sympy as sp
import scipy.stats as stats
import re
import jaconv 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import itertools
import matplotlib.dates as mdates
import pmdarima as pm
from IPython.display import clear_output
from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from kneed import KneeLocator
from sklearn.decomposition import PCA
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import time
from matplotlib.ticker import LogLocator, LogFormatter
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
from sklearnex import patch_sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='seaborn')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=UserWarning)


### 1. 課題
#### (1) データセット
- Irisのデータセット  
- MNIST（手書き数字）のデータセット
- Fashion-MNIST（衣類）のデータセット

In [16]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロードしてデータフレームを作成
def load_iris_data():
    data = load_iris()
    x = pd.DataFrame(data["data"],columns=data["feature_names"])
    y = pd.DataFrame(data["target"],columns=["target"])
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# 手書き文字のデータセットをダウンロードして、実験用データを準備
def load_mnist_data():
    data = fetch_openml('mnist_784', version=1)
    x = np.array(data['data'].astype(np.float32))
    y = np.array(data['target'].astype(np.int32))
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# Fashion-MNISTデータセットをダウンロードして、実験用データを準備
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST')
    x = np.array(data['data'].astype(np.float32))
    y = np.array(data['target'].astype(np.int32))
    x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)
    return x_train, x_test, y_train, y_test 

# 一括処理のためにデータセットの辞書を作成
dataset = {'iris': load_iris_data(), 'mnist': load_mnist_data(), 'fashon-mnist': load_fashion_mnist_data()}

#### (2) 分類モデル
##### ・k近傍法（kNN）
　説明変数に関する空間上の距離に従って，分類する手法である．ここで，$ k $は近くにある$ k $個のデータに従いクラスを決定することを意味する．$ k $が小さいとノイズに弱くなるが，大きすぎると精度が悪くなることが知られている．
##### ・サポートベクターマシン（SVM）
　対象データを2クラスに識別する手法である．一方のクラスのサポートベクターから境界への距離(マージン)と，もう一方のクラスのサポートベクターから境界への距離(マージン)が最大になる境界を採用してクラスを識別する．誤りなく識別できる場合はハードマージン法を，できない場合はソフトマージン法を利用する．
##### ・決定木（DT）
　ある説明変数についての閾値を決定し，分岐をたくさん用意することによって，分類する手法である．学習後のモデルが比較的ブラックボックスになりにくいことがメリットである．木の深さを大きくすることで制度の向上（低バイアス）が見込めるが，過学習（高バリアンス）に陥る可能性もある．
##### ・ランダムフォレスト（RF）
　バギングを用いて複数の決定木を学習させ，その多数決（アンサンブル学習）によって分類する手法である．単体の決定木よりも制度の向上が見込まれ，学習と評価は決定木が並列していることから高速に行われる．無相関な説明変数が多いと失敗する．
##### ・アダブースト（AB）
　前の学習モデルの誤りを引き継ぎ，次のモデルでその誤りが小さくなるように学習を行い，全ての学習モデルの予測結果を予測精度に従う重み付き多数決（重み付き平均）によって分類を行う手法である．ベースとなるモデルは，自由に設定可能であり，今回は決定木を用いる．
$${H(x)=\operatorname{sign}\left(\sum_{t=1}^{T} \alpha_{t} h_{t}(x)\right)}$$ 
- $H(x)$：モデルの最終出力（強学習器）
- $h(x)$：弱学習器  
- $\alpha_t$：弱学習器の重み(予測精度)  
- $\operatorname{sign}()$：SIGN関数,符号関数(正なら1,負なら-1を返す関数)  
##### ・勾配ブースティング（GBDT）
　前の学習モデルの残差誤差を引き継ぎ，次のモデルでその誤差が小さくなるように学習を行い，全ての学習モデルの予測結果を線形和で表現し分類を行う手法である．ベースとなるモデルは，決定木で固定となっており，別の弱学習器を用いてモデルを構築する場合はアダブーストから作成する必要がある．
$${F_{T}(x)=f_{0}(x)+\sum_{t=1}^{T} \alpha_{t} f_{t}(x)}$$ 
- $F_T(x)$：モデルの最終出力（強学習器）
- $f_0(x)$：初期学習器  
- $\alpha_t$：ステップサイズ(移動可能な幅)  
- $f_t(x)$：探索方向($x$ごとの探索方向を返す関数；弱学習器)  

#### (3) 条件
##### 1) k近傍法：$ k=3$
##### 2) k近傍法：$ k=3$，標準化
##### 3) SVM：kernel="linear", c=1
##### 4) SVM：kernel="linear", c=1, 標準化
##### 5) SVM：kernel="rbf", c=1
##### 6) SVM：kernel="rbf", c=1, 標準化
##### 7) 決定木：max_depth=10
##### 8) 決定木：max_depth=10, 標準化
##### 9) ランダムフォレスト：max_depth=10, n_estimators=100
##### 10) ランダムフォレスト：max_depth=10, n_estimators=100, 標準化
##### 11) アダブースト：max_depth=10, n_estimators=170
##### 12) アダブースト：max_depth=10, n_estimators=170, 標準化
##### 13) 勾配ブースティング：max_depth=10, n_estimators=170
##### 14) 勾配ブースティング：max_depth=10, n_estimators=170, 標準化

In [12]:
# 一括処理のためにモデルの辞書を作成
model = {
    # k近傍法のモデル
    'kNN(k=3)':
    KNeighborsClassifier(n_neighbors=3, # k を指定 (デフォルトは 5)
                        weights='uniform',  # 距離を考慮しない(uniform:デフォルト)、する(distance)
                        algorithm='auto', # 近傍点計算アルゴリズム (auto:デフォルト,ball_tree,kd_tree,brute)
                        leaf_size=30,  # ball_tree,kd_tree指定時のリーフサイズの設定 (デフォルトは 30)
                        p=2),  # 距離計算の次元 (2:デフォルト、1)
    # svm (kernel="linear", C=1.0) のモデル
    'SVC(kernel="linear", C=1)':
    svm.SVC(kernel="linear", C=1, max_iter=100000, verbose=True, random_state=1),
    # svm (kernel="rbf", C=1) のモデル
    'SVC(kernel="rbf", C=1)':
    svm.SVC(kernel="rbf", C=1, max_iter=100000, verbose=True, random_state=1),
    # 決定木
    'DecisionTree(max_depth=10)':
    DecisionTreeClassifier(max_depth=10, # 木の深さの最大
                            random_state=1), # 乱数シード
    # ランダムフォレストのモデル
    'randomforest(max_depth=10, n_estimators=100)':
    RandomForestClassifier(max_depth=10, # 木の深さの最大
                            n_estimators=100, # 木の数
                            random_state=1), # 乱数シード
    # アダブーストのモデル
    'Adaboost(dct(max_depth=10), n_estimators=170)':
    AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10, random_state=1), # ベースモデルを指定
                        n_estimators=170, # 木の数
                        random_state=1), # 乱数シード
    # 勾配ブースティングのモデル
    'GradientBoostingClassifier(max_depth=10, n_estimators=170)':
    GradientBoostingClassifier(max_depth=10, # 木の深さの最大
                            n_estimators=170, # 木の数
                            random_state=1), # 乱数シード
}


In [13]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    x_train, x_test, y_train, y_test = dataset[dataset_key]
    print(f'## dataset:{dataset_key} ',
        f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # データ標準化なしで性能を測定
    print('# no scaling')
    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

    # データを標準化
    print('# with scaling')
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

## dataset:iris  x_train:120 x_test:30 y_train:120 y_test:30
# no scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.95833 test_data: 0.96667
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.99167 test_data: 0.96667
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.95833 test_data: 0.96667
dataset:iris model:DecisionTree(max_depth=10) accuracy_score: train_data: 1.0 test_data: 0.96667
dataset:iris model:randomforest(max_depth=10, n_estimators=100) accuracy_score: train_data: 1.0 test_data: 0.96667
dataset:iris model:Adaboost(dct(max_depth=10), n_estimators=170) accuracy_score: train_data: 1.0 test_data: 0.96667
dataset:iris model:GradientBoostingClassifier(max_depth=10, n_estimators=170) accuracy_score: train_data: 1.0 test_data: 0.96667
# with scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.95833 test_data: 0.96667
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_

| モデル   | パラメータ                             | Iris    | MNIST   | Fashion-MNIST |
|----------|---------------------------------------|---------|---------|---------------|
| kNN      | $k=3$                                 | 0.96667 | 0.97229 | 0.8565 |
| kNN      | $k=3$，標準化                         | 0.96667 | 0.94629 | 0.85586 |
| SVM      | kernel="linear", c=1                  | 0.96667 | 0.84421 | 0.70507 |
| SVM      | kernel="linear", c=1, 標準化          | 0.96667 | 0.92214 | 0.81029 |
| SVM      | kernel="rbf", c=1                     | 0.96667 | 0.98043 | 0.89179 |
| SVM      | kernel="rbf", c=1, 標準化             | 0.96667 | 0.96607 | 0.89607 |
| DT       | max_depth=10                          | 0.96667 | 0.85386 | 0.81064 |
| DT       | max_depth=10, 標準化                  | 0.96667 | 0.85386 | 0.81043 |
| RF       | max_depth=10, n_estimators=100        | 0.96667 | 0.94879 | 0.85279 |
| RF       | max_depth=10, n_estimators=100, 標準化| 0.96667 | 0.94886 | 0.85286 |
| AB       | max_depth=10, n_estimators=170        | 0.96667 | 0.96229 | 0.85786 |
| AB       | max_depth=10, n_estimators=170, 標準化| 0.96667 | 0.96229 | 0.85786 |
| GBDT     | max_depth=10, n_estimators=170        | 0.96667 | 0.97357 | 0.89871 |
| GBDT     | max_depth=10, n_estimators=170, 標準化| 0.96667 | 0.97364 | 0.8985 |


標準化をしないと警告文が出力されることがあった．また，データセットとモデルによって標準化を行った方が精度が良くなるものと，行わない方が精度が良くなるものの両方が確認できた．興味深いことに，Irisデータではどのモデルも精度が等しくなった．これは，単純なモデルでも予測可能なほどデータがシンプルであった一方で，数個のデータに大きな矛盾があり，どのモデルにおいても上手くフィッティングができなかったものと考えられる．MNISTの結果に関しては，rbfカーネルを用い標準化を行わなかったSVMが0.98越えの高精度を記録した．これより，MNISTには非線形なデータが多く分布し，それを表現するのに最適だったものと考えられる．非線形なデータの予測に関しては，決定木でも可能であると考えられ，それを利用したRFやABも有効であると考えられるが，決定木の精度が悪く，SVMには及ばなかった．また，Fashion-MNISTの結果に関しては，標準化なしのGBDTが最良となった．GBDTに関しては，その他のデータに関しても比較的高い精度が得られ，汎用的であるといえる．

### 2. GBDTの上位互換モデルであるLightGBMの検証
　GBDTは汎用的であることが確認できたが，以下のような短所があった．
- 計算時間の長さ：各決定木を逐次的に構築するため，木の数が増えるにつれて計算時間が比較的長かった．（特に大規模データセットで影響大）
- メモリ消費の多さ：全てのデータをメモリに保持しながら学習を進めるため，大きなデータセットではメモリ使用量が高くなる可能性があった．
- 過学習のリスク：特に深い木を構築する場合，トレーニングデータに対して過学習しやすい．（適切な正則化手法を講じないとモデルの汎化性能が悪化）
- 特徴量の数が多い場合の問題：多くの特徴量が存在する場合，全ての特徴量に対してスプリットを検討するため，計算量が急増する．（高次元データに対して特徴選択や次元削減が必要）

　これらの課題を，以下のようなアルゴリズムによって克服したモデルLightGDMというものを見つけた．
- Leaf-wise（葉優先）成長戦略：最も損失を減らせる葉ノードを優先的に成長させることで，従来のDepth-wise（深さ優先）戦略に比べて少ない木の数でより高精度を実現し，全体の学習時間を短縮した．
- ヒストグラムベースのアルゴリズム：特徴量をビン（区間）に分けてヒストグラム化することで，連続的な特徴量の処理を効率化し，計算量を大幅に削減することによってメモリ使用量を押さえた．（大規模データセットでも迅速に学習可能）
- GOSS（Gradient-based One-Side Sampling）：勾配が大きいデータサンプルを優先的に選び，勾配が小さいデータの一部をランダムにサンプリングする手法によって，重要な情報を損なうことなくサンプル数を減らし，計算負荷を軽減した．
- EFB（Exclusive Feature Bundling）：疎な特徴量を束ねるEFBを利用することで次元数を減少させ，データのメモリ消費を削減し学習効率が向上した．
- 並列学習と分散学習のサポート：マルチスレッドおよび分散学習により，データを複数のスレッドやマシンに分けて処理でき，学習時間がさらに短縮された．
- Early Stopping（早期停止）：バリデーションセットを用いて早期停止を行うことが可能であり，不要な計算を省きモデルの過学習を防ぐ．

In [19]:
os.environ['LIGHTGBM_VERBOSITY'] = '0'
patch_sklearn()

for dataset_key in dataset.keys():
    x_train, x_test, y_train, y_test = dataset[dataset_key]
    print(f'## dataset:{dataset_key} ',
            f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # データ標準化なしで性能を測定
    print('# no scaling')
    
    # 学習用データを利用してLightGBMモデルを学習
    clf = LGBMClassifier(n_estimators=170, # ツリーの数
                            boosting_type='gbdt', # GBDTブースティングタイプ
                            max_depth=10,
                            verbose=0,
                            random_state=1) # 乱数シード
    clf = clf.fit(x_train, np.array(y_train).ravel())

    # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
    predict_train = clf.predict(x_train)
    train_score = metrics.accuracy_score(y_train, predict_train)
    predict_test = clf.predict(x_test)
    test_score = metrics.accuracy_score(y_test, predict_test)
    print(f'dataset:{dataset_key} model:LightGBM', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')
    
    # データを標準化
    print('# with scaling')
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    clf = LGBMClassifier(n_estimators=170, # ツリーの数
                            boosting_type='gbdt', # GBDTブースティングタイプ
                            max_depth=10,
                            verbose=0,
                            random_state=1) # 乱数シード
    clf = clf.fit(x_train_scaled, np.array(y_train).ravel())

    # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
    predict_train = clf.predict(x_train_scaled)
    train_score = metrics.accuracy_score(y_train, predict_train)
    predict_test = clf.predict(x_test_scaled)
    test_score = metrics.accuracy_score(y_test, predict_test)
    print(f'dataset:{dataset_key} model:LightGBM with scaling', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## dataset:iris  x_train:120 x_test:30 y_train:120 y_test:30
# no scaling
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

| モデル   | パラメータ                             | Iris    | MNIST   | Fashion-MNIST |
|----------|---------------------------------------|---------|---------|---------------|
| kNN      | $k=3$                                 | 0.96667 | 0.97229 | 0.8565 |
| kNN      | $k=3$，標準化                         | 0.96667 | 0.94629 | 0.85586 |
| SVM      | kernel="linear", c=1                  | 0.96667 | 0.84421 | 0.70507 |
| SVM      | kernel="linear", c=1, 標準化          | 0.96667 | 0.92214 | 0.81029 |
| SVM      | kernel="rbf", c=1                     | 0.96667 | 0.98043 | 0.89179 |
| SVM      | kernel="rbf", c=1, 標準化             | 0.96667 | 0.96607 | 0.89607 |
| DT       | max_depth=10                          | 0.96667 | 0.85386 | 0.81064 |
| DT       | max_depth=10, 標準化                  | 0.96667 | 0.85386 | 0.81043 |
| RF       | max_depth=10, n_estimators=100        | 0.96667 | 0.94879 | 0.85279 |
| RF       | max_depth=10, n_estimators=100, 標準化| 0.96667 | 0.94886 | 0.85286 |
| AB       | max_depth=10, n_estimators=170        | 0.96667 | 0.96229 | 0.85786 |
| AB       | max_depth=10, n_estimators=170, 標準化| 0.96667 | 0.96229 | 0.85786 |
| GBDT     | max_depth=10, n_estimators=170        | 0.96667 | 0.97357 | 0.89871 |
| GBDT     | max_depth=10, n_estimators=170, 標準化| 0.96667 | 0.97364 | 0.8985 |
| LightDBM | max_depth=10, n_estimators=170        | 0.96667 | 0.98057 | 0.90286 |
| LightDBM | max_depth=10, n_estimators=170, 標準化| 0.93333 | 0.98014 | 0.9035 |

結果より，LightGBMはGBDT以上に汎用的（高精度）であることが確認できた．MNISTで最良であったrbfカーネルを用いたSVMよりもLightGBMの方が若干精度が向上し，Fashion-MNISTでは0.9台を初めて突破した．反対にIrisに対しては精度が若干低下することもあった．

### 感想  
　GBDTやLightGBMがどのデータに対しても比較的高い精度で予測可能であり，比較的汎用的であることが確認できた．実用で何か分類を行う場合はLightGBMしか使われなくなったという話を読んだが，その理由がよく分かった．